<a href="https://github.com/timeseriesAI/tsai-rs" target="_parent"><img src="https://img.shields.io/badge/tsai--rs-Time%20Series%20AI%20in%20Rust-blue" alt="tsai-rs"/></a>

# Experiment Tracking with Weights & Biases (W&B)

This notebook demonstrates how to track time series experiments using **tsai-rs** with Weights & Biases.

## Purpose

Experiment tracking helps you:
1. **Compare models**: Track metrics across different architectures
2. **Reproduce results**: Log all configuration parameters
3. **Collaborate**: Share results with your team
4. **Debug**: Visualize training dynamics

## Install Dependencies

```bash
pip install wandb
cd crates/tsai_python
maturin develop --release
```

## Import Libraries

In [ ]:
import tsai_rs
import numpy as np
import wandb

print(f"tsai-rs version: {tsai_rs.version()}")
print(f"wandb version: {wandb.__version__}")
tsai_rs.my_setup()

## Login to W&B

You need to create a free account at [wandb.ai](https://wandb.ai) and get your API key.

In [ ]:
# Login to W&B (only needed once)
# wandb.login()

## Load Data

In [ ]:
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

n_vars = X_train.shape[1]
seq_len = X_train.shape[2]
n_classes = len(np.unique(y_train))

print(f"Dataset: {dsid}")
print(f"X_train shape: {X_train.shape}")
print(f"Variables: {n_vars}, Sequence length: {seq_len}, Classes: {n_classes}")

In [ ]:
# Standardize data
X_train_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)
X_test_std = tsai_rs.ts_standardize(X_test.astype(np.float32), by_sample=True)

## Create Configurable Experiment

Define what parameters you want to track.

In [ ]:
# Define experiment configuration
config = {
    'dataset': dsid,
    'architecture': 'InceptionTimePlus',
    'n_vars': n_vars,
    'seq_len': seq_len,
    'n_classes': n_classes,
    'lr': 1e-3,
    'weight_decay': 0.01,
    'batch_size': 64,
    'n_epochs': 10,
    'standardize': 'by_sample',
}

print("Experiment configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## Create tsai-rs Configuration Objects

In [ ]:
# Model configuration
model_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=config['n_vars'],
    seq_len=config['seq_len'],
    n_classes=config['n_classes']
)

# Training configuration
learner_config = tsai_rs.LearnerConfig(
    lr=config['lr'],
    weight_decay=config['weight_decay'],
    grad_clip=1.0
)

# Create datasets
train_ds = tsai_rs.TSDataset(X_train_std, y_train)
test_ds = tsai_rs.TSDataset(X_test_std, y_test)

print(f"Model config: {model_config}")
print(f"Learner config: {learner_config}")

## Track Experiment with W&B

Use `wandb.init()` to start tracking.

In [ ]:
# Example of how to structure W&B logging
# (Uncomment to run with actual W&B tracking)

# with wandb.init(project="tsai-rs-experiments", config=config, name='baseline'):
#     # Your training loop here
#     # Log metrics with: wandb.log({'accuracy': acc, 'loss': loss})
#     pass

print("W&B experiment structure:")
print("""    
with wandb.init(project="tsai-rs-experiments", config=config):
    # Training loop
    for epoch in range(n_epochs):
        # ... train ...
        wandb.log({
            'epoch': epoch,
            'train_loss': train_loss,
            'val_accuracy': val_acc
        })
""")

## Simulate Training with Logging

In [ ]:
def simulate_training(config, n_epochs=10):
    """Simulate training and return metrics for logging."""
    metrics = []
    
    for epoch in range(n_epochs):
        # Simulate improving metrics
        train_loss = 1.0 * np.exp(-epoch / 3) + np.random.normal(0, 0.05)
        val_loss = 1.1 * np.exp(-epoch / 3) + np.random.normal(0, 0.08)
        val_accuracy = 0.5 + 0.4 * (1 - np.exp(-epoch / 3)) + np.random.normal(0, 0.02)
        val_accuracy = np.clip(val_accuracy, 0, 1)
        
        epoch_metrics = {
            'epoch': epoch,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy,
        }
        metrics.append(epoch_metrics)
        
        print(f"Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, val_acc={val_accuracy:.4f}")
    
    return metrics

# Run simulated training
print("Simulated training run:")
print("-" * 60)
metrics = simulate_training(config)

## Compare Multiple Architectures

In [ ]:
# Define architectures to compare
architectures = {
    'InceptionTimePlus': tsai_rs.InceptionTimePlusConfig(
        n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
    ),
    'ResNetPlus': tsai_rs.ResNetPlusConfig(
        n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
    ),
    'TST': tsai_rs.TSTConfig(
        n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
    ),
    'MiniRocket': tsai_rs.MiniRocketConfig(
        n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
    ),
}

print(f"{'Architecture':<20} {'Config Type'}")
print("-" * 50)
for name, config_obj in architectures.items():
    print(f"{name:<20} {type(config_obj).__name__}")

In [ ]:
# Simulate comparison across architectures
def run_architecture_comparison(architectures):
    """Run experiments for multiple architectures."""
    results = {}
    
    for arch_name in architectures.keys():
        print(f"\nRunning: {arch_name}")
        print("-" * 40)
        
        # Simulate final accuracy (different baselines for different architectures)
        base_acc = {
            'InceptionTimePlus': 0.88,
            'ResNetPlus': 0.85,
            'TST': 0.90,
            'MiniRocket': 0.86,
        }
        
        final_acc = base_acc.get(arch_name, 0.80) + np.random.normal(0, 0.03)
        final_acc = np.clip(final_acc, 0, 1)
        
        results[arch_name] = {
            'final_accuracy': final_acc,
            'epochs': 10,
        }
        
        print(f"  Final accuracy: {final_acc:.4f}")
    
    return results

comparison_results = run_architecture_comparison(architectures)

## Hyperparameter Sweeps with W&B

W&B supports automated hyperparameter sweeps.

In [ ]:
# Example sweep configuration
sweep_config = {
    'method': 'bayes',  # or 'grid', 'random'
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'lr': {
            'values': [1e-4, 1e-3, 1e-2]
        },
        'weight_decay': {
            'values': [0.0, 0.01, 0.1]
        },
        'architecture': {
            'values': ['InceptionTimePlus', 'ResNetPlus', 'TST']
        },
    }
}

print("Example W&B sweep configuration:")
import json
print(json.dumps(sweep_config, indent=2))

## Logging Best Practices

What to log for time series experiments:

In [ ]:
# Comprehensive logging example
experiment_log = {
    # Dataset info
    'dataset': dsid,
    'n_samples_train': len(X_train),
    'n_samples_test': len(X_test),
    'n_vars': n_vars,
    'seq_len': seq_len,
    'n_classes': n_classes,
    
    # Preprocessing
    'standardize': 'by_sample',
    'augmentation': ['gaussian_noise', 'mag_scale'],
    
    # Model
    'architecture': 'InceptionTimePlus',
    
    # Training
    'lr': 1e-3,
    'weight_decay': 0.01,
    'batch_size': 64,
    'n_epochs': 10,
    'scheduler': 'OneCycleLR',
    
    # Results
    'best_val_accuracy': 0.91,
    'final_test_accuracy': 0.89,
}

print("Recommended experiment log fields:")
print("-" * 50)
for key, value in experiment_log.items():
    print(f"  {key}: {value}")

## Visualize Results

In [ ]:
import matplotlib.pyplot as plt

# Visualize architecture comparison
arch_names = list(comparison_results.keys())
accuracies = [comparison_results[a]['final_accuracy'] for a in arch_names]

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(arch_names, accuracies, color='steelblue', edgecolor='black')

# Add value labels
for bar, acc in zip(bars, accuracies):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
            f'{acc:.2%}', ha='center', va='bottom')

ax.set_ylabel('Accuracy')
ax.set_title(f'Architecture Comparison on {dsid}')
ax.set_ylim(0, 1)
ax.axhline(y=np.mean(accuracies), color='red', linestyle='--', label=f'Mean: {np.mean(accuracies):.2%}')
ax.legend()

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated experiment tracking concepts with tsai-rs:

### Key Points
1. **Configuration management**: Define all parameters upfront
2. **W&B integration**: Log metrics, configs, and artifacts
3. **Architecture comparison**: Systematically evaluate models
4. **Hyperparameter sweeps**: Automate search for best parameters

### W&B Integration Pattern
```python
import wandb

config = {
    'dataset': dsid,
    'architecture': 'InceptionTimePlus',
    'lr': 1e-3,
    # ... other params
}

with wandb.init(project="tsai-rs", config=config):
    # Training loop
    wandb.log({'accuracy': acc, 'loss': loss})
```

### tsai-rs Configuration
```python
model_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
)
learner_config = tsai_rs.LearnerConfig(
    lr=1e-3, weight_decay=0.01
)
```

In [ ]:
# Quick reference
print("W&B + tsai-rs Quick Reference")
print("=" * 50)
print("\n# Initialize W&B")
print("wandb.init(project='tsai-rs', config=config)")
print("\n# Log metrics")
print("wandb.log({'accuracy': acc, 'loss': loss})")
print("\n# tsai-rs model config")
print("config = tsai_rs.InceptionTimePlusConfig(n_vars, seq_len, n_classes)")
print("\n# Create sweep")
print("sweep_id = wandb.sweep(sweep_config, project='tsai-rs')")